créditos:https://colab.research.google.com/drive/1FvBBw2JNXHy9WeJusI2PPr2PMnGVbbGM#scrollTo=GBotcBrgqR30 (lembrar de referenciar)

In [53]:
#%matplotlib inline
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
#carregando o dataset
data_set = '/content/drive/My Drive/WESAD/'
s2_path = data_set + 'S2.pkl'
#modificação necessária pois os dados estavam em py2 e o ambiente usa py3
with open(s2_path, 'rb') as file:
    s2_data = pickle.load(file, encoding='latin1')

In [75]:
#Separando dados de peito de dados de pulso (apenas dados de peito serão considerados conforme análise dos dados)
c_ax=s2_data['signal']['chest']['ACC'][0:,0]
c_ay=s2_data['signal']['chest']['ACC'][0:,1]
c_az=s2_data['signal']['chest']['ACC'][0:,2]
c_ecg=s2_data['signal']['chest']['ECG'][:,0]
c_emg=s2_data['signal']['chest']['EMG'][:,0]
c_eda=s2_data['signal']['chest']['EDA'][:,0]
c_temp=s2_data['signal']['chest']['Temp'][:,0]
c_resp=s2_data['signal']['chest']['Resp'][:,0]
label=s2_data['label']

# Alterar o objeto de dicionário para um array numpy e o array para um dataframe

*(referenciar)*
**Python | Numpy ndarray.T**
https://www.geeksforgeeks.org/python-numpy-ndarray-t/

**Gerando e Manipulando Arrays em Python**
https://towardsdatascience.com/numpy-array-cookbook-generating-and-manipulating-arrays-in-python-2195c3988b09

In [ ]:
peito = [c_ax, c_ay, c_az, c_ecg, c_eda, c_emg, c_resp, c_temp, label]
p_array = np.array(peito)
p_array = p_array.T #(transposição matriz)
Colunas = ['c_ax', 'c_ay', 'c_az', 'c_ecg', 'c_eda', 'c_emg', 'c_resp', 'c_temp', 'label']

p_df = pd.DataFrame(p_array, columns = Colunas)
p_df

In [ ]:
p_df.describe().T

Identificando os rótulos, usando apenas:
neutro: 0
stress: 1
diversão: 2

Os demais serão desconsiderados nesse estudo

In [ ]:
neutro = p_df.loc[p_df['label'] == 0]
stress = p_df.loc[p_df['label'] == 1]
diversao = p_df.loc[p_df['label'] == 2]

p_loc = pd.concat([neutro, stress, diversao])
p_loc

# Normalizando dados: train test

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_axis.html (referenciar)

In [82]:
y = p_loc.label
x = p_loc.drop ('label', axis= 1)
x_train, x_test, y_train, y_test = train_test_split (x, y, train_test_split = 0.2)
print(x_train.shape)
print(y_test.shape)
evalSet = [(x_train, y_train), (x_test, y_test)]

TypeError: ignored